In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 ## only use 50 % of the gpu
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

###  Importing  Necessary Library

In [2]:
import numpy as np
from glob import glob
from tensorflow.keras.layers import (Input, Lambda, Dense, Flatten)
from tensorflow.keras.models import (Model, Sequential)
from tensorflow.keras.applications.mobilenet_v2 import (MobileNetV2, preprocess_input)
from tensorflow.keras.preprocessing.image import (ImageDataGenerator, load_img, array_to_img, img_to_array)
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'dataset/train'
valid_path = 'dataset/val'
test_path =  'dataset/test'

In [4]:
# Import the MobileNet library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

mobile_net  = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
print("No Of The Layer In MObileNet :--> ", len(mobile_net.layers))

No Of The Layer In MObileNet :-->  154


In [6]:
  # useful for getting number of output classes
folders = glob('dataset/train/*')
print("total no of the folder in the train dataset :-->", len(folders))


total no of the folder in the train dataset :--> 100


In [7]:
###  Data Preprocessing

In [8]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2))

test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
# Make sure to provide the same target size as initialied for the image size

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

validation_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 1200 images belonging to 100 classes.
Found 400 images belonging to 100 classes.


In [10]:
print("LIST OF THE CLASS NAME  ARE :-", list(training_set.class_indices.keys()))

LIST OF THE CLASS NAME  ARE :- ['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [11]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(mobile_net.layers))

# Fine-tune from this layer onwards
fine_tune_at = 80

# Freeze all the layers before the `fine_tune_at` layer
for layer in mobile_net.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  154


In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

#learning_rate reduce module
lr_reduce = ReduceLROnPlateau('val_loss', patience=4, 
                                              factor=0.5, min_lr=1e-6)

# Stop early if model doesn't improve after n epochs
early_stopper = EarlyStopping(monitor='val_loss', patience=4,
                              verbose=0, restore_best_weights=True)

In [1]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Define a more complex Sequential model
model = Sequential()

# Add MobileNetV2 base model
model.add(mobile_net)

# Add additional convolutional block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Add Dropout layer
model.add(Dropout(0.5))  # Example: dropout rate of 0.5

# Add fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Example: dropout rate of 0.5
model.add(Dense(256, activation='relu'))

# Output layer
model.add(Dense(len(folders), activation='softmax'))

NameError: name 'Sequential' is not defined

In [14]:
model.compile(optimizer=Adam(1e-5),  # Low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 512)               32113152  
                                                                 
 dense_1 (Dense)             (None, 100)               51300     
                                                                 
Total params: 34,422,436
Trainable params: 34,203,236
Non-trainable params: 219,200
_________________________________________________________________


In [16]:
history_fine = model.fit(training_set,
                         batch_size = 32,
                         epochs=50,
                         validation_data=validation_set,
                         callbacks=[lr_reduce, early_stopper],
                         verbose=1, shuffle=True)

C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\keras\preprocessing\image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/50
38/38 [==============================] - ETA: 0s - loss: 5.2503 - accuracy: 0.0117

In [ ]:
import matplotlib.pyplot as plt
# plot the loss
plt.plot(history_fine.history['loss'], label='train loss')
plt.plot(history_fine.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history_fine.history['accuracy'], label='train acc')
plt.plot(history_fine.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('MobileNet.h5')

In [ ]:
y_pred = np.argmax(model.predict(validation_set), axis =1)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model2=load_model('MobileNet.h5')

In [ ]:
img=image.load_img('Datasets/Test/Coffee/download (2).jpg',target_size=(224,224))


In [ ]:
x=image.img_to_array(img)
x=x/255


In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)


In [ ]:
a=np.argmax(model.predict(img_data), axis=1)
a